In [1]:
from pymongo import MongoClient
import re
from elasticsearch import Elasticsearch,helpers
from pprint import pprint
import pickle
from pyjarowinkler.distance import get_jaro_distance
import editdistance
import fuzzy

In [2]:
def findES_Doc(entityName):
#     "from" : 0, "size" : 10,
    return  {
#         "size": 10,
        "query":
        {

            "match":  { "stdName": entityName }

        }
    }

In [3]:
def something(collection, collName):
    # dt_to_state = load('DTCode_to_State')
    cursor = collection.find({"$or":[{'entities':{"$exists":True}}, {'districtsLocation':{"$exists":True}}]}).batch_size(50)
    states = dict()
    for article in cursor:
        distinct_states = set()
        if 'entities' in article:
            for state_entity in article['entities']:
                if state_entity['type'] == 'ProvinceOrState':
                    distinct_states.add(state_entity['name'].upper())

            for state in distinct_states:
                if state in states:
                    states[state] += 1
                else:
                    states[state] = 1

    cursor.close()

    return states

In [4]:
with open("./state_code_state_name", 'rb') as f:
    state_info=pickle.load(f)
state_names = list(map(lambda x: x.lower(), list(state_info.values())))
state_names

['jammu & kashmir',
 'himachal pradesh',
 'punjab',
 'chandigarh',
 'uttarakhand',
 'haryana',
 'nct of delhi',
 'rajasthan',
 'uttar pradesh',
 'bihar',
 'sikkim',
 'arunachal pradesh',
 'nagaland',
 'manipur',
 'mizoram',
 'tripura',
 'meghalaya',
 'assam',
 'west bengal',
 'jharkhand',
 'odisha',
 'chhattisgarh',
 'madhya pradesh',
 'gujarat',
 'daman & diu',
 'dadra & nagar haveli',
 'maharashtra',
 'andhra pradesh',
 'karnataka',
 'goa',
 'lakshadweep',
 'kerala',
 'tamil nadu',
 'puducherry',
 'andaman & nicobar islands']

In [5]:


# *******************************************************************************************
es = Elasticsearch('10.237.26.117', port=9200, timeout=30)

#create connection to mongo

client = MongoClient('localhost', 27017)
db = client['media-db2']

collName = input("Enter Collection Name: ")
collection = db[collName]

# check if connected to elasticSearch
if es.ping():
    print("connected to ES")
else:
    print("Connection to ES failed")


es_index='index-19apr20-r'

es_mapping='mapping-19apr20-r'  
#*************************************************************************************************

# raw_data = es.indices.get_mapping( es_index )
# print ("get_mapping() response type:", type(raw_data))
# pprint(raw_data)

locations = something(collection, collName)

Enter Collection Name: agriculture_schemes
connected to ES


In [6]:
# print(list(states.keys())[156])
# res = es.search(index=es_index, body=findES_Doc(list(states.keys())[156]))
# pprint(res)

In [7]:
def pheonetic_distance(name1, name2):

    ''' this returns edit distance for phonetic similarity for two words'''
    soundness1 = fuzzy.nysiis(name1)
    soundness2 = fuzzy.nysiis(name2)
    nysiis_score = editdistance.eval(soundness1, soundness2)

    return nysiis_score, soundness1, soundness2

In [8]:
def match(name1, name2):
    lgth = min(len(name1), len(name2))
    
    jaro_score = get_jaro_distance(name1, name2)
#         print("Jaro : " , jaro_score)
    editDist_score = editdistance.eval(name1, name2)
#         print("Edit : ", editDist_score)
    sound_score, sond1, sond2 = pheonetic_distance(name1, name2)
#         print("soundness : ", sound_score)
    s_lgth = min(len(sond1), len(sond2))

    if (lgth <= 8  and jaro_score >= 0.94) or (lgth > 8 and  lgth <= 12 and jaro_score >= 0.9) or (lgth > 12 and jaro_score >= 0.87):
#             print("Case 1" )
        return True

    elif (lgth <= 3 and editDist_score < 1) or (lgth > 3 and lgth <= 8 and editDist_score < 2) or (lgth > 8 and editDist_score <= 3):
#             print("Case 2")
        return True

    elif (s_lgth ==0) or (s_lgth <= 3 and sound_score < 1) or (s_lgth > 3 and s_lgth <= 8 and sound_score < 2) or (s_lgth > 8 and sound_score <= 3):
#             print("Case 3")
        return True
    return False

In [9]:
def match_state(cand):
    global state_names
    for state in state_names:
        if state in cand:
            return state
        if match(cand, state):
            return state
    return None

In [21]:
for loc in locations:
    loc = loc.lower()
    res = es.search(index=es_index, body=findES_Doc(loc))
    print(loc, end = ': ')
    hits = res['hits']['hits']
    for i in range(len(hits)):
        candidate = []
        if 'resolutions' in hits[i]['_source'] and hits[i]['_source']['resolutions'] != None and 'containedbycountry' in hits[i]['_source']['resolutions'] and hits[i]['_source']['resolutions']['containedbycountry'] == 'India':
            candidate.append(hits[i]['_source']['resolutions']['containedbystate'])
        if 'aliases' in hits[i]['_source']:
            candidate += hits[i]['_source']['aliases']
        breaked = False
        for cand in candidate:
            state = match_state(cand.lower())
            if state != None:
                print(state, end = '')
                breaked = True
                break
        if breaked:
            break
    print()

andhra pradesh: andhra pradesh
goa: goa
tamil nadu: tamil nadu
karnataka: karnataka
maryland: 
hawaii: 
kerala: kerala
new mexico: 
assams jorhat: assam
west bengal: west bengal
himachal pradesh: himachal pradesh
madhya pradesh: madhya pradesh
yadgir: 
tirupur: tripura
virudhunagar: 
kodagu: karnataka
coimbatore: 
gujarat: gujarat
sholapur: 
maharashtra: maharashtra
madurai: 
gulbarga: 
raichur: karnataka
kannur: 
namakkal: 
uttar pradesh: uttar pradesh
orissa: odisha
jharkhand: jharkhand
bihar: bihar
dubai: 
assams nagaon: assam
rayagada: 
rayagada district: 
thiruvalla: 
betul: 
sikkim: sikkim
ranga reddy: 
tripura: tripura
mizoram: mizoram
mamit: mizoram
karur: kerala
tiruvarur: 
theni: 
cuddalore: 
krishna: 
anantapur: 
dakshina kannada district: 
madurai district: 
bjp: 
ganjam: odisha
assam: assam
rajasthan: rajasthan
haryana: haryana
vizianagaram district: 
jammu and kashmir: jammu & kashmir
kashmir: 
nilgiris district: 
trialsthe coimbatore district: 
hyderabad: 
manipur: manip

ernakulam district: kerala
sangmner district: 
chittur-thathamangalam: 
central calcutta: 
dk: 
wisconsin: 
bahraich: 
qinghai: 
gajapati: gujarat
gajapti: 
yadagir: 
barmer: 
ernakulam: 
nagpur district: 
khurda: 
panchkula: 
thiruvananthapuram district: 
warrington: 
antique: 
p.m.mysore district: 
pakur: 
bardhaman: 
tournamentthe krishna district: 
madhya pradeshs mandsaur: madhya pradesh
chickballapur: 
malabar: 
calicut: 
bidhan nagar municipality: 
baluchistan: 
jhalawar: 
azamgarh: 
pathanamthitta: 
nuton,general manager telecom district: manipur
general manager telecom district: manipur
khandwa: 
guntur district: 
banjar: 
deogarh: odisha
odishas sambalpur: 
jajarkot: 
shamli: 
sindh: 
sonebhadra: 
mondaygulbarga: 
sundargarh: chandigarh
mayurbhanj: odisha
udham singh nagar: 
kannauj: 
jalore: 
ards: 
raipur: tripura
pathanamthitta district: 
aleppo: 
vijayapura: 
raqqa: 
uttarakhands tehri: uttarakhand
manang: 
tonk: rajasthan
balangir: 
pali: 
sirohi: 
hurdlethe udupi: 
tiru

scarborough: 
faisalabad: 
gpms: 
rajasthan and gujarat: rajasthan
bathinda: 
barrackpore municipality: 
surajpur: 
sindhupalchowk: 
dolakha: 
rotherham: 
sonepur: manipur
mahesana: 
east siang: arunachal pradesh
hanumangarh: rajasthan
shandong: 
valais: 
west midnapore: west bengal
banke: west bengal
papersthe: 
sipahijala: 
sulu: 
kanpur: 
pilibhit: 
ghumusar: 
chakdah: 
sabarkantha: gujarat
alpes-maritimes: 
shillong: 
kinnaur: himachal pradesh
congress-run behrampore: 
sakarkantha: 
aceh: 
solan: sikkim
udaipur: rajasthan
neb: 
aravalli: kerala
vadodara: 
vehari: 
percentageamritsar: 
rewa: assam
kathua: 
ct: 
kashmirs shopian: sikkim
jind: haryana
mohali: 
patiala: 
south 24-parganas: 
malerkotla: 
bhid: 
parbhani: 
tdp prakasam: 
valsad: 
nrhm: punjab
parishad krishna: 
had ernakulum: kerala
ernakuam: 
bengaluru: 
u11: jammu & kashmir
ropar: tripura
ambikapur: 
thrikkakara: 
bagalkote: 
istria: 
nirmal: 
barabanki: 
mainpuri: manipur
del: 
mattannur: 
kujambail: 
narmada: 
bihars

bjps: 
keralas mahapuran: kerala
schepmans: 
brussels: 
islam: 
sukma district: sikkim
mt: 
hanumanghari: 
south kashmirs anantnag: 
south kashmirs shopian: sikkim
daraa: 
jackson county: 
hasiru sene: 
tirunelveli city: 
keralas kannur: kerala
3,714sad statisticsernakulam: 
north kashmirs baramulla: sikkim
lok satta udyama samstha: 
sindhupalchok: 
badaun district: 
pithoragarh: uttarakhand
hawaii county: 
bandipora: 
yuva morcha: 
bihars darbhanga: bihar
uri: 
bharuch district: 
south kashmirs kulgam: 
jharkhands latehar: jharkhand
bihars vaishali: bihar
sambalpur municipality: 
gir somnath: 
budhipadar: 
icu.the: 
nepals banke: west bengal
dhenkanl: 
aydin: assam
bodoland: 
a. shaikmohideenthe: jammu & kashmir
jac: 
nakhon si thammarat: chandigarh
kandy: chandigarh
hebei: 
jilin: 
kanke: 
kharan: haryana
dera bugti: 
awaran: 
lasbela: 
kech: 
marion county: haryana
florida.'insults: 
deir el-zour: kerala
fugu county: 
shaanxi province: 
monmouth: 
costa county: 
closedthe mandya: 
p

mosul: 
veracruz: 
taez: 
mati: 
maizhokunggar: 
oman: 
ziarat: 
gujrat: gujarat
dhanbagh: 
e: jammu & kashmir
telanganas khammam: 
la: 
bihars bhagalpur: bihar
bihars madhepura: manipur
uttar pradeshs varanasi: uttar pradesh
odishas balasore: odisha
jharkhands palamu: jharkhand
chutara: 
pakistans punjab: punjab
punjabs barnala: punjab
gomati: 
kamalpur: 
tamil nadus thiruvallur: tamil nadu
branchesdakshina kannada: 
ri-bhoi: 
jowai: meghalaya
delhis: 
kashmirs kishtwar: sikkim
dengue,rajkot: 
akshay thakur: 
dr b r ambedkar: 
upper assams tinsukia: assam
karnataka and andhra pradesh: andhra pradesh
bannu: 
cmcs: 
manhattan: 
nuwakot district: 
irans hormozgan province: sikkim
baluch: 
khorasan: 
gopalgunj: 
birnagar: 
as-sams karbi anglong: kerala
odishas sundergarh: odisha
srinagar: 
assams chirang: assam
bundu municipality: 
bihars rohtas: bihar
sir m visvesvaraya district: 
arizona and colorado in southwestern united states, california: 
california and us: jammu & kashmir
etah dis

thrissur-ernakulam: 
sho.hyv mainpuri: manipur
brts: 
koriya: 
sunaria: jammu & kashmir
kirkuk: 
parsons green: 
zhashui county: 
chhapra: 
north hamgyong province: 
freedomby neel mukherjee: 
patialas rajpura: tripura
south kanara: goa
kanara: jammu & kashmir
negro: 
street vendors association: odisha
andhra pradeshs srikakulam: andhra pradesh
bihars champaran: bihar
telegraph picturepatna: 
roing: 
east singhbum: west bengal
south sikkim: sikkim
chiba prefecture: 
wenzhous ouhai: 
coxs bazar: 
saurashtras bhavnagar: maharashtra
mustang district: 
odishas rayagada: odisha
maharashtras osmanabad: maharashtra
anantapuram: 
perumbavoor: 
tripolis upmarket ben ashour: tripura
chinas xinjiang: 
mexico city: 
tlalpan: 
asir: assam
bihars west champaran: bihar
sitapurs: 
chengalpet: 
puja the east singhbhum: west bengal
kmc borough: 
bihars supaul: bihar
hyogo: 
nagappattinam: 
jammu-kashmirs anantnag: 
andhra pradeshs nellore: andhra pradesh
euphrates province: 
mass: 
bihars gopalganj: bih

facebooka jind: haryana
punjabs faridkot: punjab
rtm kottayam: 
ofthanjavur: 
thodupuzha district: 
bihars katihar: bihar
maradu: 
kashmirs baramulla: 
jalaun district: 
purba barddhaman: 
suryapet: 
farah province: 
vijayanagar: 
vijayanagara: 
barwala: 
jharkhands dumka: jharkhand
istthe shahjahanpur: 
peddapalli: 
bagalakot: 
manesar: manipur
mayurbhanj distjharsuguda: odisha
dakshina kannda: 
dharwad district: 
punjabs mohali: punjab
momand dara: 
tarn taran district: 
beed districtin beed: 
coorg: 
greater chennai district: 
manhattans west village: 
sri ganganagar: 
narowal: kerala
maharashtra and gujarat: gujarat
bengals murshidabad: west bengal
amethis: 
kanhangad: 
uttar pradeshs saharanpur: uttar pradesh
states malkangiri: 
wuhan: 
chinas hubei province: 
punjabs tarn taran: punjab
baodi: 
hubei province: 
keralaevery: kerala
chiba: 
columbia circuit: 
karakax county: 
hilly: 
north eastern: 
cheongdo: 
chinas hubei: 
haryana and western uttar pradesh: haryana
sfi ernakulam: 